Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (80000, 28, 28) (80000,)
Validation set (4000, 28, 28) (4000,)
Test set (4000, 28, 28) (4000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (80000, 784) (80000, 10)
Validation set (4000, 784) (4000, 10)
Test set (4000, 784) (4000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

ANS: 

Without regulation, test accuracy is 86%.

Adding L2 regulation with coefficient 1e-3, valid accuracy is 82.2% and test accuracy is 88.5%.

In [25]:
###SGD
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  print(logits)
  l2regulate= tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases)
  print(l2regulate)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+1e-3*l2regulate)
  print(loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Tensor("add:0", shape=(128, 10), dtype=float32)
Tensor("add_1:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.702599
Minibatch accuracy: 12.5%
Validation accuracy: 11.9%
Minibatch loss at step 500: 2.519578
Minibatch accuracy: 79.7%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 2.126567
Minibatch accuracy: 72.7%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 1.256612
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 0.831731
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 0.860380
Minibatch accuracy: 76.6%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 0.655830
Minibatch accuracy: 81.2%
Validation accuracy: 82.3%
Test accuracy: 88.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [16]:
###SGD
train_subsettt = train_dataset[0:1000,:]
train_sublabel = train_labels[0:1000,:]
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_subsettt.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_subsettt[offset:(offset + batch_size), :]
    batch_labels = train_sublabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.158688
Minibatch accuracy: 14.1%
Validation accuracy: 17.8%
Minibatch loss at step 200: 0.795997
Minibatch accuracy: 85.2%
Validation accuracy: 68.7%
Minibatch loss at step 400: 0.197597
Minibatch accuracy: 96.9%
Validation accuracy: 68.7%
Minibatch loss at step 600: 0.091157
Minibatch accuracy: 99.2%
Validation accuracy: 69.0%
Minibatch loss at step 800: 0.055331
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Minibatch loss at step 1000: 0.051629
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 1200: 0.037718
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1400: 0.032254
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Minibatch loss at step 1600: 0.033336
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 1800: 0.030272
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Minibatch loss at step 2000: 0.022581
Minibatch accuracy: 100.0%
Validation ac

ANS: training accuracy is 100%, but validation accuracy and test accuracy is around 70%.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

ANS: 
With doptout probability 0.8, overfitting is reduced but still high.
Valid accuracy and test accuracy is about 76%, similar to that with without dropout.
Ithome says that this method is useful for large NN. Maybe my NN is too small...

In [34]:
###SGD
train_subsettt = train_dataset[0:1000,:]
train_sublabel = train_labels[0:1000,:]
batch_size = 128
dddropout=[0.4, 0.6, 0.8, 1.0]
num_steps = 3001
for dropouttt in dddropout:
    graph = tf.Graph()
    with graph.as_default():
      # Input data. 
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
      keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
      # Variables.
      weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))
      
      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      logits = tf.nn.dropout(logits, dropouttt)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
        
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_subsettt.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_subsettt[offset:(offset + batch_size), :]
        batch_labels = train_sublabel[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropouttt}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict = {keep_prob:1.0}), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 29.629913
Minibatch accuracy: 7.0%
Validation accuracy: 10.8%
Minibatch loss at step 1000: 1.148076
Minibatch accuracy: 51.6%
Validation accuracy: 60.6%
Minibatch loss at step 2000: 1.267884
Minibatch accuracy: 41.4%
Validation accuracy: 61.2%
Minibatch loss at step 3000: 1.213020
Minibatch accuracy: 53.1%
Validation accuracy: 61.8%
Test accuracy: 67.1%
Initialized
Minibatch loss at step 0: 25.094456
Minibatch accuracy: 4.7%
Validation accuracy: 5.8%
Minibatch loss at step 1000: 0.582349
Minibatch accuracy: 71.1%
Validation accuracy: 61.4%
Minibatch loss at step 2000: 0.558712
Minibatch accuracy: 68.8%
Validation accuracy: 62.4%
Minibatch loss at step 3000: 0.603332
Minibatch accuracy: 69.5%
Validation accuracy: 63.1%
Test accuracy: 69.2%
Initialized
Minibatch loss at step 0: 18.107578
Minibatch accuracy: 15.6%
Validation accuracy: 14.4%
Minibatch loss at step 1000: 0.279734
Minibatch accuracy: 83.6%
Validation accuracy: 63.9%
Minibatch loss at ste

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---

In [5]:
###SGD
train_subsettt = train_dataset[0:40000,:]
train_sublabel = train_labels[0:40000,:]
batch_size = 128
dropouttt=0.8

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.07, global_step, 2000, 0.9, staircase=True)
  keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
  # Variables.
  ###weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  ###biases = tf.Variable(tf.zeros([num_labels]))
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 128], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([128]))
  weights2 = tf.Variable(tf.truncated_normal([128, 128], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([128]))
  weights3 = tf.Variable(tf.truncated_normal([128, 128], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([128]))
  weights4 = tf.Variable(tf.truncated_normal([128, num_labels]))
  biases4 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  ###logits = tf.matmul(tf_train_dataset, weights) + biases
  trainlayer1= tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  trainlayer2= tf.nn.relu(tf.matmul(trainlayer1, weights2) + biases2)
  trainlayer3= tf.nn.relu(tf.matmul(trainlayer2, weights3) + biases3)
  logits = tf.matmul(trainlayer3, weights4)+biases4
  logits = tf.nn.dropout(logits, dropouttt)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  validlayer1= tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  validlayer2= tf.nn.relu(tf.matmul(validlayer1, weights2) + biases2)
  validlayer3= tf.nn.relu(tf.matmul(validlayer2, weights3) + biases3)
  logitsvalid= tf.matmul(validlayer3, weights4) + biases4
  testlayer1= tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  testlayer2= tf.nn.relu(tf.matmul(testlayer1, weights2) + biases2)
  testlayer3= tf.nn.relu(tf.matmul(testlayer2, weights3) + biases3)
  logitstest= tf.matmul(testlayer3, weights4) + biases4

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logitsvalid)#(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(logitstest)#(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_subsettt.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_subsettt[offset:(offset + batch_size), :]
    batch_labels = train_sublabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropouttt}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict = {keep_prob:1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 8.245696
Minibatch accuracy: 15.6%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.508271
Minibatch accuracy: 84.4%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 0.390574
Minibatch accuracy: 86.7%
Validation accuracy: 86.7%
Minibatch loss at step 6000: 0.318987
Minibatch accuracy: 86.7%
Validation accuracy: 86.9%
Minibatch loss at step 8000: 0.279548
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Minibatch loss at step 10000: 0.264300
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 12000: 0.331534
Minibatch accuracy: 81.2%
Validation accuracy: 86.7%
Minibatch loss at step 14000: 0.238445
Minibatch accuracy: 85.9%
Validation accuracy: 87.0%
Minibatch loss at step 16000: 0.239847
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 18000: 0.150504
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 20000: 0.216310
Minibatch accuracy: 89.1%
Validation 